<a href="https://colab.research.google.com/github/PaulinaTarkowsk/sql_recap/blob/main/Dash_multipage_exercise_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 17.9 MB/s eta 0:00:00


In [2]:
from dash import Dash, dcc, html, callback, Input, Output
import pandas as pd
import numpy as np
import plotly.express as px
import threading
from google.colab import output, drive

In [5]:
drive.mount('/content/drive')
population = pd.read_csv('/content/drive/My Drive/Dash/Aplikacja_multipage/population.csv')
new_books = pd.read_csv("/content/drive/My Drive/Dash/Aplikacja_multipage/new-books-per-million.csv")
new_books = new_books[(new_books["Year"]>=1940) & (new_books["Year"]<=1996)]
selected_books = new_books[new_books["Year"]==1996][["Entity", "Book titles per capita (Fink-Jensen 2015)"]]
selected_pops = population[population["Year"]==1996][["Entity", "Population (historical estimates)"]]
data = pd.merge(selected_books, selected_pops, how="inner", on="Entity")

Mounted at /content/drive


In [6]:
scatter_figure = px.scatter(data_frame=data,
                 x="Book titles per capita (Fink-Jensen 2015)",
                 y="Population (historical estimates)",
                 text="Entity",
                 log_x=True,
                 log_y=True,
                 template="none",
                 title="Country population vs number of new book titles per capita in 1996")
scatter_figure.update_traces(textposition="bottom right")

In [22]:
layout_page_three = html.Div([
    dcc.Graph(id='population-graph'),
    dcc.Slider(
        id='num-countries-slider',
        min=5,
        max=30,
        step=5,
        value=10,
        marks={i: str(i) for i in range(5, 31, 5)},
    ),
])

In [7]:
layout_page_two = html.Div(
    children=[
              dcc.Graph(
                  id='scatter',
                  figure=scatter_figure
                  ),
              dcc.Graph(id='line')
              ],
)

In [17]:
layout_page_one = html.Div(
    children=[
              dcc.Graph(
                  id='some-lines',
                  ),
              dcc.Input(
            id='country-input',
            type='text',
            placeholder='Enter country name',
            #value='Switzerland'
        )
              ],
)

In [42]:
app = Dash()

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),

    dcc.Link('Linechart with dropdown', href='/'),
    html.Br(),
    dcc.Link('Cross filtering in dash', href='/second_page'),
    html.Br(),
    dcc.Link('Population graph', href='/third_page'),

    html.Div(id='page-content')
])

@callback(
    Output('page-content', 'children'),
    Input('url', 'pathname'))
def display_page(pathname):



  if pathname == "/third_page":
    return layout_page_three
  elif pathname == "/second_page":
    return layout_page_two
  else:
    return layout_page_one

@app.callback(
    Output('line', 'figure'),
    Input('scatter','hoverData')
)
def build_line_chart(country):

  selected_country = country["points"][0]["text"]
  selected_data = new_books[new_books["Entity"]==selected_country]

  fig = px.line(data_frame=selected_data,
                x="Year",
                y="Book titles per capita (Fink-Jensen 2015)",
                template="none",
                title="Book titles per capita in %s for years 1940-1996" % selected_country
                )
  return fig

@app.callback(
    Output('some-lines', 'figure'),
    Input('country-input', 'value')
)
def build_county_line(value):

  present_books = new_books[new_books["Entity"]==value]

  fig = px.line(data_frame=present_books,
              x="Year",
              y="Book titles per capita (Fink-Jensen 2015)",
              color="Entity",
              template="none",
              title="Book titles per capita in different countries for years 1940-1996"
              )
  fig.update_layout(showlegend=False)
  return fig

@app.callback(
    Output('population-graph', 'figure'),
    [Input('num-countries-slider', 'value')]
)
def update_figure(selected_num):
   bar_data = selected_pops.sort_values(by="Population (historical estimates)", ascending = False)
   bar_data = bar_data[(bar_data.Entity != "World")  &
                      (bar_data.Entity != "Asia") &
                      (bar_data.Entity != "Africa") &
                      (bar_data.Entity != "Europe") &
                      (bar_data.Entity != "North America") &
                      (bar_data.Entity != "South America")]
   bar_data = bar_data.head(selected_num)
   fig = px.bar(bar_data, x='Entity', y='Population (historical estimates)', title='Ranking państw pod względem populacji w 1996 roku')



   return fig

In [43]:
thread = threading.Thread(target=app.run_server)
thread.start()
output.serve_kernel_port_as_window(port=8050)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>